# Lab 6: Decorators + Object-Oriented Python

## Overview

In today's lecture, which mostly covered rules, definitions, and semantics of OOP, we'll be playing around with actual classes today, writing a fair chunk of code and building several classes to solve a variety of problems.

Recall our starting definitions:

- An *object* has identity
- A *name* is a reference to an object
- A *namespace* is an associative mapping from names to objects
- An *attribute* is any name following a dot ('.')

## Building Decorators

Recall that a decorator is a special type of function that accepts a function as an argument and returns a new function which (usually) wraps some of the behavior of the supplied function.

Furthermore, recall that the `@decorator` syntax is syntactic sugar.

```Python
@decorator
def fn():
    pass
```

is equivalent to

```Python
def fn():
    pass
fn = decorator(fn)
```


### Review

In lecture, we implemented the `debug` decorator.

```Python
def debug(function):
    def wrapper(*args, **kwargs):
        print("Arguments:", args, kwargs)
        return function(*args, **kwargs)
    return wrapper
```

Take a moment, with a partner, and make sure you understand what is happening in the above lines. Why are the arguments to wrapper on the second line `*args` and `**kwargs` instead of something else? What would happen if we didn't `return wrapper` at the end of the function body?

### Automatic Caching (20 points)
Write a decorator `cache` that will automatically cache any calls to the decorated function. You can assume that all arguments passed to the decorated function will always be hashable types.
```Python
def cache(function):
    pass  # Your implementation here
```

In pseudocode, to accomplish this you will
```
take in some function f
build a new function g: when called with some arguments...
    if we have seen these arguments before:
        return a saved result for these arguments
    otherwise:
        compute and return result of calling f with these arguments and save the result in some data structure
return g
```

For example, you should be able to use this decorator as follows:

```Python
@cache
def fib(n):
    return fib(n-1) + fib(n-2) if n > 2 else 1

fib(10)  # 55 (takes a moment to execute)
fib(10)  # 55 (returns immediately)
fib(100) # doesn't take forever
fib(400) # doesn't raise RuntimeError
```
Hint: You can set arbitrary attributes on a function (e.g. `fn._cache`). When you do so, the attribute-value pair also gets inserted into `fn.__dict__`. Take a look for yourself. Are the extra attributes and `.__dict__` always in sync?

In [51]:
from functools import *

def cache(function):
    @wraps(function)
    @lru_cache(None)
    def run(*args, **kwargs):
        function(*args,**kwargs)
        return function(*args,**kwargs)
    return run


@cache
def fib(n):
    return fib(n-1) + fib(n-2) if n > 2 else 1

print(fib(10))  # 55 (takes a moment to execute)
print(fib(10))  # 55 (returns immediately)
print(fib(100)) # doesn't take forever
print(fib(400)) # doesn't raise RuntimeError

55
55
354224848179261915075
176023680645013966468226945392411250770384383304492191886725992896575345044216019675


## Courses

### Basic Class (5 points)

Let’s create a class to represent courses at NUK! A course will have three attributes to start: a department (like `"CS"`), a course code (like `"41"` or `"92SI"`), and a title (like `"Python"`).

```Python
class Course:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
```

You can assume that all arguments to this constructor will be strings.

Running the following code cell will create a class object `Course` and print some information about it.

*Note: If you change the content of this class definition, you will need to re-execute the code cell for it to have any effect. Any instance objects of the old class object will not be automatically updated, so you may need to rerun instantiations of this class object as well.*

In [12]:
class Course:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        
print(Course)
print(Course.mro())
print(Course.__init__)

<class '__main__.Course'>
[<class '__main__.Course'>, <class 'object'>]
<function Course.__init__ at 0x0000000005CA9BF8>


We create an instance of the class by instantiating the class object, supplying some arguments.

```Python
python = Course("CS", "41", "the python programming language")
```

#### Print out the three attributes of the `python` instance object. 

In [13]:
python = Course("CS", "41", "the python programming language")

print(python.department)  # Print out the department of python
print(python.code)  # Print out the code of python
print(python.title)  # Print out the title of python

CS
41
the python programming language


### Inheritance

Let's explore inheritance by creating a `CSCourse` class that takes an additional parameter `recorded` that defaults to `False`.

In [14]:
class CSCourse(Course):
    def __init__(self, department, code, title, recorded=False):
        super().__init__(department, code, title)
        self.is_recorded = recorded

We haven't seen the `super()` call yet, and it's mostly just magic, but it concretely lets us treat the `self` object as an instance object of the immediate superclass (as measured by MRO), so we can call the superclass's `__init__` method.

We can instantiate our new class:

```Python
a = Course("CS", "106A", "Programming Methodology")
b = CSCourse("CS", "106B", "Programming Abstractions")
x = CSCourse("CS", "106X", "Programming Abstractions", recorded=True)
print(a.code)  # => "106A"
print(b.code)  # => "106B"
```

Read through the following statements and try to predict their output.

```Python
type(a)
isinstance(a, Course)
isinstance(b, Course)
isinstance(x, Course)
isinstance(x, CSCourse)
issubclass(x, CSCourse)
issubclass(Course, CSCourse)
type(a) == type(b)
type(b) == type(x)
a == b
b == x
```

In [ ]:
a = Course("CS", "106A", "Programming Methodology")
b = CSCourse("CS", "106B", "Programming Abstractions")
x = CSCourse("CS", "106X", "Programming Abstractions", recorded=True)

print(type(a))
print(isinstance(a, Course))
print(isinstance(b, Course))
print(isinstance(x, Course))
print(isinstance(x, CSCourse))
print(issubclass(Course, CSCourse))
print(type(a) == type(b))
print(type(b) == type(x))
print(a == b)
print(b == x)

### Additional Attributes (10 points)

Let's add more functionality to the `Course` class!

* Add a attribute `students` to the instances of the `Course` class that tracks whether students are present. Initially, students should be an empty set.
* Create a method `mark_attendance(*students)` that takes a variadic number of `students` and marks them as present.
* Create a method `is_present(student)` that takes a student’s name as a parameter and returns `True` if the student is present and `False` otherwise.

In [6]:
class Course:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        self.students = []
    def mark_attendance(self, *students):
        self.students.extend(list(students))
    
    def is_present(self, student):
        return student in self.students

    
python = Course("CS", "41", "the python programming language")
python.mark_attendance("Bob", "John", "Kevin", "Mason")
python.mark_attendance("Alice", "Kristen")
print(python.is_present("Kristen")) # => True
print(python.is_present("Brad")) # => False

True
False


### Implementing Prerequisites (10 points)

Now, we'll focus on `CSCourse`. We want to implement functionality to determine if one computer science course is a prerequisite of another. In our implementation, we will assume that the ordering for courses is determined first by the numeric part of the course code: for example, `140` comes before `255`. If there is a tie, the ordering is determined by the default string ordering of the letters that follow. For example, `106A < 106B`. After implementing, you should be able to see:

```Python
>>> cs106a = CSCourse("CS", "106A", "Programming Methodology")
>>> cs106b = CSCourse("CS", "106B", "Programming Abstractions")
>>> cs107 = CSCourse("CS", "107", "Computer Organzation and Systems")
>>> cs110 = CSCourse("CS", "110", "Principles of Computer Systems")
>>> cs110 > cs106b
True
>>> cs107 > cs110
False
```

To accomplish this, you will need to implement a magic method `__le__` that will add functionality to determine if a course is a prerequisite for another course. Read up on [total ordering](https://docs.python.org/3/library/functools.html#functools.total_ordering) to figure out what `__le__` should return based on the argument you pass in.

To give a few hints on how to add this piece of functionality might be implemented, consider how you might extract the actual `int` number from the course code attribute.

Additionally, you should implement a `__eq__` on `Course`s. Two classes are equivalent if they are in the same department and have the same course code: the course title doesn't matter here.

In [82]:
class Course:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
    
    def __le__(self, other):
        return self.code <= other.code
    
    def __eq__(self, other):
        return self.code == other.code

#### Sorting

Now that we've written a `__le__` method and an `__eq__` method, we've implemented everything we need to speak about an "ordering" of `Course`s. Using the [`functools.total_ordering` decorator](https://docs.python.org/3/library/functools.html#functools.total_ordering), decorate the class so that all of the comparison methods are implemented. You should be able to run

In [83]:
import functools
@functools.total_ordering
class CSCourse(Course):
    def __init__(self, department, code, title, recorded=False):
        super().__init__(department, code, title)
        self.is_recorded = recorded

# Let's make CS106A a CS course
cs106a = CSCourse("CS", "106A", "Programming Methodology")
cs106b = CSCourse("CS", "106B", "Programming Abstractions")
cs107 = CSCourse("CS", "107", "Computer Organzation and Systems")
cs110 = CSCourse("CS", "110", "Principles of Computer Systems")

courses = [cs110, cs106a, cs107, cs106b]
courses.sort()
print([c.code for c in courses]) # => ['106A', '106B', '107', '110']

['106A', '106B', '107', '110']


## Inheritance

Consider the following code:

```Python
"""Examples of Single Inheritance"""
class Transportation:
    wheels = 0

    def __init__(self):
        self.wheels = -1

    def travel_one(self):
        print("Travelling on generic transportation")

    def travel(self, distance):
        for _ in range(distance):
            self.travel_one()

    def is_auto(self):
        return self.wheels == 4

class Bike(Transportation):

    def travel_one(self):
        print("Biking one mile")

class Car(Transportation):
    wheels = 4

    def travel_one(self):
        print("Driving one mile")

    def make_sound(self):
        print("VROOM")

class Ferrari(Car):
    pass

t = Transportation()
b = Bike()
c = Car()
f = Ferrari()
```

Predict the outcome of each of the following lines of code.

```Python
isinstance(t, Transportation)

isinstance(b, Bike)
isinstance(b, Transportation)
isinstance(b, Car)
isinstance(b, t)

isinstance(c, Car)
isinstance(c, Transportation)

isinstance(f, Ferrari)
isinstance(f, Car)
isinstance(f, Transportation)

issubclass(Bike, Transportation)
issubclass(Car, Transportation)
issubclass(Ferrari, Car)
issubclass(Ferrari, Transportation)
issubclass(Transportation, Transportation)

b.travel(5)
c.is_auto()
f.is_auto()
b.is_auto()
b.make_sound()
c.travel(10)
f.travel(4)
```

In [40]:
class Transportation:
    wheels = 0

    def __init__(self):
        self.wheels = -1

    def travel_one(self):
        print("Travelling on generic transportation")

    def travel(self, distance):
        for _ in range(distance):
            self.travel_one()

    def is_auto(self):
        return self.wheels == 4

class Bike(Transportation):

    def travel_one(self):
        print("Biking one mile")

class Car(Transportation):
    wheels = 4

    def travel_one(self):
        print("Driving one mile")

    def make_sound(self):
        print("VROOM")

class Ferrari(Car):
    pass

t = Transportation()
b = Bike()
c = Car()
f = Ferrari()

In [41]:
print(isinstance(t, Transportation))
print('----------------')

print(isinstance(b, Bike))
print(isinstance(b, Transportation))
print(isinstance(b, Car))
print(isinstance(b, type(Car)))
print('----------------')

print(isinstance(c, Car))
print(isinstance(c, Transportation))
print('----------------')

print(isinstance(f, Ferrari))
print(isinstance(f, Car))
print(isinstance(f, Transportation))
print('----------------')

print(issubclass(Bike, Transportation))
print(issubclass(Car, Transportation))
print(issubclass(Ferrari, Car))
print(issubclass(Ferrari, Transportation))
print(issubclass(Transportation, Transportation))
print('----------------')

b.travel(5)
print(c.is_auto())
print(f.is_auto())
print(b.is_auto())
# b.make_sound()
print('----------------')

c.travel(10)
f.travel(4)

True
----------------
True
True
False
False
----------------
True
True
----------------
True
True
True
----------------
True
True
True
True
True
----------------
Biking one mile
Biking one mile
Biking one mile
Biking one mile
Biking one mile
False
False
False
----------------
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile


## Magic Methods

### Reading

Python provides an enormous number of special methods that a class can override to interoperator with builtin Python operations. You can skim through an [approximate visual list](http://diveintopython3.problemsolving.io/special-method-names.html) from Dive into Python3, or a [more verbose explanation](https://rszalski.github.io/magicmethods/), or the [complete Python documentation](https://docs.python.org/3/reference/datamodel.html#specialnames) on special methods. Fair warning, there are a lot of them, so it's probably better to skim than to really take a deep dive, unless you're loving this stuff.

### Writing (Polynomial Class) (20 points)

We will write a `Polynomial` class that acts like a number. As a a reminder, a [polynomial](https://en.wikipedia.org/wiki/Polynomial) is a mathematical object that looks like $1 + x + x^2$ or $4 - 10x + x^3$ or $-4 - 2x^{10}$. A mathematical polynomial can be evaluated at a given value of $x$. For example, if $f(x) = 1 + x + x^2$, then $f(5) = 1 + 5 + 5^2 = 1 + 5 + 25 = 31$.

Polynomials are also added componentwise: If $f(x) = 1 + 4x + 4x^3 + 2x^4$ and $g(x) = 2 + 3x^2 + 5x^3$, then $(f + g)(x) = (1 + 2) + 4x + 3x^2 + (4 + 5)x^3 + 2x^4 = 3 + 4x + 3x^2 + 9x^3 + 2x^4$.

Construct a polynomial with a variadic list of coefficients: the zeroth argument is the coordinate of the $x^0$'s place, the first argument is the coordinate of the $x^1$'s place, and so on. For example, `f = Polynomial(1, 3, 5)` should construct a `Polynomial` representing $1 + 3x + 5x^2$.

You will need to override the addition special method (`__add__`) and the callable special method (`__call__`).

You should be able to emulate the following code:

```Python
f = Polynomial(1, 5, 10)
g = Polynomial(1, 3, 5)

print(f(5))  # => Invokes `f.__call__(5)`
print(g(2))  # => Invokes `g.__call__(2)`

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`
```

Lastly, implement a method to convert a `Polynomial` to an informal string representation. For example, the polynomial `Polynomial(1, 3, 5)` should be represented by the string `"1 * x^0 + 3 * x^1 + 5 * x^2"`.

In [87]:
from itertools import *
class Polynomial:
    def __init__(self ,*args):
        self.ls = list(args)
        pass
    
    def __call__(self, x):
        count = 0
        sum = 0
        for i in self.ls:
            sum += x ** count * i
            count += 1
        return sum
            
            
        """Implement `self(x)`."""
    
    def __add__(self, other):
        tmp = Polynomial()
        tmp.ls = [x + y for x, y in zip_longest(self.ls,other.ls,fillvalue=0)]
        return tmp
        """Implement `self + other`."""
    
    def __str__(self):
        str1=''
        for i ,y in zip(self.ls,range(len(self.ls))):
            if y:
                str1 += ' + '
            str1 = str1 + str(i) + " * x^" + str(y)
        return str1
        """Implement `str(x)`."""
    

f = Polynomial(1, 5, 10)
g = Polynomial(1, 3, 5, 1)

print(f(5))  # => Invokes `f.__call__(5)`, output 276
print(g(2))  # => Invokes `g.__call__(2)`, output 35

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`, output 188

print(h)     # => 2 * x^0 + 8 * x^1 + 15 * x^2 + 1 * x^3

276
35
188
2 * x^0 + 8 * x^1 + 15 * x^2 + 1 * x^3


> With <3 by @sredmond